In [1]:
# script to collect all of the thermo calculations and put them together into a database

In [2]:
import os
import re
import glob
from rmgpy.data.thermo

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [6]:
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
thermo_libs = glob.glob(os.path.join(DFT_DIR, 'thermo', 'species*', 'arkane', 'RMG_libraries'))

In [42]:
# Load the Arkane thermo
entries = []
for i, lib_path in enumerate(thermo_libs):
    matches = re.search('species_([0-9]{4})', lib_path)
    species_index = int(matches[1])
    ark_thermo_database = rmgpy.data.thermoThermoDatabase()
    ark_thermo_database.load_libraries(
        lib_path,
    )

    for key in ark_thermo_database.libraries['thermo'].entries.keys():
        entry = ark_thermo_database.libraries['thermo'].entries[key]
        entry.index = species_index
        entries.append(entry)


In [55]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_thermo_database = rmgpy.data.thermoThermoDatabase()
ark_thermo_database.libraries['thermo'] = rmgpy.data.thermo.ThermoLibrary()
ark_thermo_database.libraries['thermo'].label = 'harris_butane'
ark_thermo_database.libraries['thermo'].entries = OrderedDict()
for entry in entries:
    ark_thermo_database.libraries['thermo'].entries[entry.label] = entry

In [56]:
# save the results
output_path = os.path.join(DFT_DIR, 'thermo')
ark_thermo_database.save_libraries(output_path)